42226

14226

In [36]:
import pickle


def get_unique_dict(dictionary_1, dictionary_2):
    dictionary = {}

    for key in range(42226):
        if key < 28000:
            dictionary[key] = dictionary_1[key]
        else:
            dictionary[key] = dictionary_2[key]

    return dictionary

with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_train_0-28000.p', 'rb') as f:
    dictionary_1 = pickle.load(f)

with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_train_28000-42000.p', 'rb') as f:
    dictionary_2 = pickle.load(f)

# merge the two dictionaries
dictionary = get_unique_dict(dictionary_1, dictionary_2)


# save the dictionary
# with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_train.p', 'wb') as f:
#     pickle.dump(dictionary, f)

# BASELINE with yolov5l6

In this assessment, we are analyzing the baseline performance. The YOLOv5l6 model was utilized to locate the boxes, while the corresponding text for these boxes was computed in advance using CLIP.

In [28]:
import torch
import torch.nn as nn
import numpy as np
import torchvision

import pickle
from tqdm import tqdm

In [31]:

import pickle


def get_unique_dict(dictionary_1, dictionary_2):
    dictionary = {}

    for key in range(42226):
        if key < 28000:
            dictionary[key] = dictionary_1[key]
        else:
            dictionary[key] = dictionary_2[key]

    return dictionary

with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_train_0-28000.p', 'rb') as f:
    dictionary_1 = pickle.load(f)

with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_train_28000-42000.p', 'rb') as f:
    dictionary_2 = pickle.load(f)

# merge the two dictionaries
dictionary = get_unique_dict(dictionary_1, dictionary_2)


def load_the_best_matching_box(dictionary):
    box_target = []
    box_pred = []
    
    # with open(file_name, 'rb') as f:
    #     dictionary = pickle.load(f)
        
    for sample in tqdm(dictionary.keys()):
        
        try:
            # get the sample
            element = dictionary[sample]
            
            for sim, cap in zip(element['text_similarity'],element['caption']):
                
                # get the max similarity index
                max_sim_index = torch.argmax(sim).numpy()
                
                # gete the  bbox_target
                box_pred.append(element['df_boxes'].loc[max_sim_index][:4].values.astype('int'))
                box_target.append(element['bbox_target'])
        except:
            KeyError: print('error in sample:', sample)
            
    box_pred = torch.stack([torch.Tensor(p) for p in box_pred])
    box_target = torch.tensor(box_target)
    
    # convert the box_pred to x1, y1, x2, y2
    box_target[:, 2] = box_target[:, 0] + box_target[:, 2]
    box_target[:, 3] = box_target[:, 1] + box_target[:, 3]
        
    return box_pred, box_target



box_pred, box_target = load_the_best_matching_box(dictionary)



print(box_pred,box_target)


print('assert same dimensionality',len(box_pred) == len(box_target))



print(box_target.shape,  box_pred.shape)



mean_iou = []
accuracy_0_3 = []
accuracy_0_5 = []
accuracy_0_75 = []

def iou(box1, box2):
    # get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1
    b2_x1, b2_y1, b2_x2, b2_y2 = box2
    
    # get the corrdinates of the intersection rectangle
    inter_rect_x1 =  max(b1_x1, b2_x1)
    inter_rect_y1 =  max(b1_y1, b2_y1)
    inter_rect_x2 =  min(b1_x2, b2_x2)
    inter_rect_y2 =  min(b1_y2, b2_y2)
    
    # Intersection area
    inter_area = max(inter_rect_x2 - inter_rect_x1 + 1, 0) * max(inter_rect_y2 - inter_rect_y1 + 1, 0)
    
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou
    
for b_t, b_p in zip(box_target, box_pred):
    
    IoU = iou(b_t, b_p)

    # get the iou
    mean_iou.append(IoU)

    # get the accuracy
    if IoU > 0.3:
        accuracy_0_3.append(1)
    else:
        accuracy_0_3.append(0)

    if IoU > 0.5:
        accuracy_0_5.append(1)
    else:
        accuracy_0_5.append(0)

    if IoU > 0.75:
        accuracy_0_75.append(1)
    else:   
        accuracy_0_75.append(0)

# get the mean iou
mean_iou_m = torch.stack(mean_iou).mean().numpy()


# get the accuracy
accuracy_0_3_m = np.array(accuracy_0_3).mean()
accuracy_0_5_m = np.array(accuracy_0_5).mean()
accuracy_0_75_m = np.array(accuracy_0_75).mean()

# get precision
precision_0_3 = np.array(accuracy_0_3).sum() / len(accuracy_0_3)
precision_0_5 = np.array(accuracy_0_5).sum() / len(accuracy_0_5)
precision_0_75 = np.array(accuracy_0_75).sum() / len(accuracy_0_75)

# get recall
recall_0_3 = np.array(accuracy_0_3).sum() / len(box_target)
recall_0_5 = np.array(accuracy_0_5).sum() / len(box_target)
recall_0_75 = np.array(accuracy_0_75).sum() / len(box_target)


# mean iou on the train set
print('mean iou on the train set:',mean_iou_m)

# accuracy on the train set
print('accuracy on the train set:', 'T=0.3',accuracy_0_3_m, 'T=0.5',accuracy_0_5_m, 'T=0.75',accuracy_0_75_m)

# precision on the train set
print('precision on the train set:', 'T=0.3',precision_0_3, 'T=0.5',precision_0_5, 'T=0.75',precision_0_75)

# recall on the train set
print('recall on the train set:', 'T=0.3',recall_0_3, 'T=0.5',recall_0_5, 'T=0.75',recall_0_75)



100%|██████████| 42226/42226 [00:11<00:00, 3813.78it/s]


tensor([[135.,  36., 304., 224.],
        [  0.,  42., 235., 465.],
        [281., 486., 399., 576.],
        ...,
        [376.,  89., 487., 379.],
        [  0., 245.,  48., 399.],
        [ 30.,  65., 382., 638.]]) tensor([[  0.0000,  45.9500, 238.9200, 454.5900],
        [  0.0000,  45.9500, 238.9200, 454.5900],
        [213.7200, 456.5100, 405.7300, 590.2600],
        ...,
        [141.3400,  86.7100, 256.6700, 375.4600],
        [ 31.7100,  67.0300, 384.0000, 640.0000],
        [ 31.7100,  67.0300, 384.0000, 640.0000]])
assert same dimensionality True
torch.Size([72601, 4]) torch.Size([72601, 4])
mean iou on the train set: 0.52733845
accuracy on the train set: T=0.3 0.567967383369375 T=0.5 0.5243178468616134 T=0.75 0.49508959931681384
precision on the train set: T=0.3 0.567967383369375 T=0.5 0.5243178468616134 T=0.75 0.49508959931681384
recall on the train set: T=0.3 0.567967383369375 T=0.5 0.5243178468616134 T=0.75 0.49508959931681384


In [32]:

import pickle

with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_test.p', 'rb') as f:
    dictionary = pickle.load(f)


def load_the_best_matching_box(dictionary):
    box_target = []
    box_pred = []
    
    # with open(file_name, 'rb') as f:
    #     dictionary = pickle.load(f)
        
    for sample in tqdm(dictionary.keys()):
        
        try:
            # get the sample
            element = dictionary[sample]
            
            for sim, cap in zip(element['text_similarity'],element['caption']):
                
                # get the max similarity index
                max_sim_index = torch.argmax(sim).numpy()
                
                # gete the  bbox_target
                box_pred.append(element['df_boxes'].loc[max_sim_index][:4].values.astype('int'))
                box_target.append(element['bbox_target'])
        except:
            KeyError: print('error in sample:', sample)
            
    box_pred = torch.stack([torch.Tensor(p) for p in box_pred])
    box_target = torch.tensor(box_target)
    
    # convert the box_pred to x1, y1, x2, y2
    box_target[:, 2] = box_target[:, 0] + box_target[:, 2]
    box_target[:, 3] = box_target[:, 1] + box_target[:, 3]
        
    return box_pred, box_target



box_pred, box_target = load_the_best_matching_box(dictionary)



print(box_pred,box_target)


print('assert same dimensionality',len(box_pred) == len(box_target))



print(box_target.shape,  box_pred.shape)



mean_iou = []
accuracy_0_3 = []
accuracy_0_5 = []
accuracy_0_75 = []

def iou(box1, box2):
    # get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1
    b2_x1, b2_y1, b2_x2, b2_y2 = box2
    
    # get the corrdinates of the intersection rectangle
    inter_rect_x1 =  max(b1_x1, b2_x1)
    inter_rect_y1 =  max(b1_y1, b2_y1)
    inter_rect_x2 =  min(b1_x2, b2_x2)
    inter_rect_y2 =  min(b1_y2, b2_y2)
    
    # Intersection area
    inter_area = max(inter_rect_x2 - inter_rect_x1 + 1, 0) * max(inter_rect_y2 - inter_rect_y1 + 1, 0)
    
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou
    
for b_t, b_p in zip(box_target, box_pred):
    
    IoU = iou(b_t, b_p)

    # get the iou
    mean_iou.append(IoU)

    # get the accuracy
    if IoU > 0.3:
        accuracy_0_3.append(1)
    else:
        accuracy_0_3.append(0)

    if IoU > 0.5:
        accuracy_0_5.append(1)
    else:
        accuracy_0_5.append(0)

    if IoU > 0.75:
        accuracy_0_75.append(1)
    else:   
        accuracy_0_75.append(0)

# get the mean iou
mean_iou_m = torch.stack(mean_iou).mean().numpy()


# get the accuracy
accuracy_0_3_m = np.array(accuracy_0_3).mean()
accuracy_0_5_m = np.array(accuracy_0_5).mean()
accuracy_0_75_m = np.array(accuracy_0_75).mean()

# get precision
precision_0_3 = np.array(accuracy_0_3).sum() / len(accuracy_0_3)
precision_0_5 = np.array(accuracy_0_5).sum() / len(accuracy_0_5)
precision_0_75 = np.array(accuracy_0_75).sum() / len(accuracy_0_75)

# get recall
recall_0_3 = np.array(accuracy_0_3).sum() / len(box_target)
recall_0_5 = np.array(accuracy_0_5).sum() / len(box_target)
recall_0_75 = np.array(accuracy_0_75).sum() / len(box_target)


# mean iou on the train set
print('mean iou on the train set:',mean_iou_m)

# accuracy on the train set
print('accuracy on the train set:', 'T=0.3',accuracy_0_3_m, 'T=0.5',accuracy_0_5_m, 'T=0.75',accuracy_0_75_m)

# precision on the train set
print('precision on the train set:', 'T=0.3',precision_0_3, 'T=0.5',precision_0_5, 'T=0.75',precision_0_75)

# recall on the train set
print('recall on the train set:', 'T=0.3',recall_0_3, 'T=0.5',recall_0_5, 'T=0.75',recall_0_75)



100%|██████████| 5023/5023 [00:01<00:00, 3565.75it/s]


tensor([[374.,  69., 510., 262.],
        [374.,  69., 510., 262.],
        [  0., 171.,  22., 331.],
        ...,
        [202., 277., 334., 445.],
        [224.,   0., 346., 137.],
        [224.,   0., 346., 137.]]) tensor([[374.3100,  65.0600, 510.3500, 267.0000],
        [374.3100,  65.0600, 510.3500, 267.0000],
        [ 93.9500,  83.2900, 598.5600, 373.8600],
        ...,
        [348.5800, 230.9100, 480.0000, 471.1400],
        [212.8200,   0.0000, 355.9000, 132.4100],
        [212.8200,   0.0000, 355.9000, 132.4100]])
assert same dimensionality True
torch.Size([8595, 4]) torch.Size([8595, 4])
mean iou on the train set: 0.52096236
accuracy on the train set: T=0.3 0.5647469458987784 T=0.5 0.5161140197789412 T=0.75 0.48632926119837117
precision on the train set: T=0.3 0.5647469458987784 T=0.5 0.5161140197789412 T=0.75 0.48632926119837117
recall on the train set: T=0.3 0.5647469458987784 T=0.5 0.5161140197789412 T=0.75 0.48632926119837117


In [33]:

import pickle

with open('./data/yolo_v8x/yolo_v8x_1_dictionary_full_val.p', 'rb') as f:
    dictionary = pickle.load(f)


def load_the_best_matching_box(dictionary):
    box_target = []
    box_pred = []
    
    # with open(file_name, 'rb') as f:
    #     dictionary = pickle.load(f)
        
    for sample in tqdm(dictionary.keys()):
        
        try:
            # get the sample
            element = dictionary[sample]
            
            for sim, cap in zip(element['text_similarity'],element['caption']):
                
                # get the max similarity index
                max_sim_index = torch.argmax(sim).numpy()
                
                # gete the  bbox_target
                box_pred.append(element['df_boxes'].loc[max_sim_index][:4].values.astype('int'))
                box_target.append(element['bbox_target'])
        except:
            KeyError: print('error in sample:', sample)
            
    box_pred = torch.stack([torch.Tensor(p) for p in box_pred])
    box_target = torch.tensor(box_target)
    
    # convert the box_pred to x1, y1, x2, y2
    box_target[:, 2] = box_target[:, 0] + box_target[:, 2]
    box_target[:, 3] = box_target[:, 1] + box_target[:, 3]
        
    return box_pred, box_target



box_pred, box_target = load_the_best_matching_box(dictionary)



print(box_pred,box_target)


print('assert same dimensionality',len(box_pred) == len(box_target))



print(box_target.shape,  box_pred.shape)



mean_iou = []
accuracy_0_3 = []
accuracy_0_5 = []
accuracy_0_75 = []

def iou(box1, box2):
    # get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1
    b2_x1, b2_y1, b2_x2, b2_y2 = box2
    
    # get the corrdinates of the intersection rectangle
    inter_rect_x1 =  max(b1_x1, b2_x1)
    inter_rect_y1 =  max(b1_y1, b2_y1)
    inter_rect_x2 =  min(b1_x2, b2_x2)
    inter_rect_y2 =  min(b1_y2, b2_y2)
    
    # Intersection area
    inter_area = max(inter_rect_x2 - inter_rect_x1 + 1, 0) * max(inter_rect_y2 - inter_rect_y1 + 1, 0)
    
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou
    
for b_t, b_p in zip(box_target, box_pred):
    
    IoU = iou(b_t, b_p)

    # get the iou
    mean_iou.append(IoU)

    # get the accuracy
    if IoU > 0.3:
        accuracy_0_3.append(1)
    else:
        accuracy_0_3.append(0)

    if IoU > 0.5:
        accuracy_0_5.append(1)
    else:
        accuracy_0_5.append(0)

    if IoU > 0.75:
        accuracy_0_75.append(1)
    else:   
        accuracy_0_75.append(0)

# get the mean iou
mean_iou_m = torch.stack(mean_iou).mean().numpy()


# get the accuracy
accuracy_0_3_m = np.array(accuracy_0_3).mean()
accuracy_0_5_m = np.array(accuracy_0_5).mean()
accuracy_0_75_m = np.array(accuracy_0_75).mean()

# get precision
precision_0_3 = np.array(accuracy_0_3).sum() / len(accuracy_0_3)
precision_0_5 = np.array(accuracy_0_5).sum() / len(accuracy_0_5)
precision_0_75 = np.array(accuracy_0_75).sum() / len(accuracy_0_75)

# get recall
recall_0_3 = np.array(accuracy_0_3).sum() / len(box_target)
recall_0_5 = np.array(accuracy_0_5).sum() / len(box_target)
recall_0_75 = np.array(accuracy_0_75).sum() / len(box_target)


# mean iou on the train set
print('mean iou on the train set:',mean_iou_m)

# accuracy on the train set
print('accuracy on the train set:', 'T=0.3',accuracy_0_3_m, 'T=0.5',accuracy_0_5_m, 'T=0.75',accuracy_0_75_m)

# precision on the train set
print('precision on the train set:', 'T=0.3',precision_0_3, 'T=0.5',precision_0_5, 'T=0.75',precision_0_75)

# recall on the train set
print('recall on the train set:', 'T=0.3',recall_0_3, 'T=0.5',recall_0_5, 'T=0.75',recall_0_75)



100%|██████████| 2573/2573 [00:00<00:00, 3149.33it/s]


tensor([[275.,  26., 429., 152.],
        [275.,  26., 429., 152.],
        [308., 172., 499., 395.],
        ...,
        [238.,   0., 383., 258.],
        [183.,  65., 430., 419.],
        [183.,  65., 430., 419.]]) tensor([[285.0400,  23.2900, 424.7800, 146.4000],
        [285.0400,  23.2900, 424.7800, 146.4000],
        [183.4000,  68.3400, 310.2100, 167.1500],
        ...,
        [238.3800,   5.4700, 388.3100, 257.8700],
        [177.8600,  67.1400, 430.8400, 419.5600],
        [177.8600,  67.1400, 430.8400, 419.5600]])
assert same dimensionality True
torch.Size([4369, 4]) torch.Size([4369, 4])
mean iou on the train set: 0.53202575
accuracy on the train set: T=0.3 0.5781643396658275 T=0.5 0.5326161593041886 T=0.75 0.4998855573357748
precision on the train set: T=0.3 0.5781643396658275 T=0.5 0.5326161593041886 T=0.75 0.4998855573357748
recall on the train set: T=0.3 0.5781643396658275 T=0.5 0.5326161593041886 T=0.75 0.4998855573357748


In [37]:
file_name = './data/yolo_v8x/yolo_v8x_1_dictionary_full_test.p'

with open(file_name, 'rb') as f:
    dictionary = pickle.load(f)

print('maximum number of boxes found: ',max([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]))
print('mean number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).mean())
print('std number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).std())
print('count max number of boxes found: ',[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(32))
print('count of number of boxes found:')
print([[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(xx+1) for xx in range(32)])

maximum number of boxes found:  48
mean number of boxes found:  7.110292653792555
std number of boxes found:  4.077023158130561
count max number of boxes found:  0
count of number of boxes found:
[2, 55, 522, 757, 823, 708, 527, 350, 274, 190, 187, 152, 125, 72, 54, 49, 34, 34, 16, 17, 11, 16, 15, 8, 7, 4, 2, 1, 3, 1, 0, 0]


In [38]:
file_name = './data/yolo_v8x/yolo_v8x_1_dictionary_full_val.p'

with open(file_name, 'rb') as f:
    dictionary = pickle.load(f)

print('maximum number of boxes found: ',max([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]))
print('mean number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).mean())
print('std number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).std())
print('count max number of boxes found: ',[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(32))
print('count of number of boxes found:')
print([[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(xx+1) for xx in range(32)])

maximum number of boxes found:  33
mean number of boxes found:  7.382821609016712
std number of boxes found:  4.111538592521283
count max number of boxes found:  0
count of number of boxes found:
[0, 35, 227, 400, 409, 294, 262, 189, 153, 132, 102, 74, 67, 62, 43, 26, 15, 17, 22, 8, 12, 4, 6, 3, 0, 9, 1, 0, 0, 0, 0, 0]


In [39]:
file_name = './data/yolo_v8x/yolo_v8x_1_dictionary_full_train.p'

with open(file_name, 'rb') as f:
    dictionary = pickle.load(f)

print('maximum number of boxes found: ',max([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]))
print('mean number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).mean())
print('std number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).std())
print('count max number of boxes found: ',[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(32))
print('count of number of boxes found:')
print([[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(xx+1) for xx in range(32)])

maximum number of boxes found:  47
mean number of boxes found:  7.235423672618766
std number of boxes found:  4.153021557423283
count max number of boxes found:  13
count of number of boxes found:
[18, 479, 4468, 5996, 6630, 5439, 4523, 3301, 2520, 2061, 1476, 1100, 856, 653, 635, 504, 348, 231, 234, 176, 114, 62, 73, 51, 65, 45, 32, 24, 19, 8, 16, 13]
